In [1]:
import pandas as pd
import numpy as np 
from chembl_webresource_client.new_client import new_client
from rdkit import Chem
from rdkit.Chem import Descriptors, Lipinski

In [2]:
# Target search function
def target_search(target_name):
    target = new_client.target
    res = target.filter(pref_name__icontains = target_name)
    if len(res) > 200: #Prevents large df from taking too long to load in the app
        raise ValueError('Search Target too broad, Try Another Search Term.')
    else:
        targets = pd.DataFrame(res)
        selection = ['organism', 'pref_name', 'target_chembl_id', 'target_type']
        return(targets[selection]) #Returns a df of targets matching your search term
    

In [3]:
df_target_search = target_search("HIV")

In [4]:
def get_bioactivities_data(target_chembl_ID):
    activity = new_client.activity
    res = activity.filter(target_chembl_id=target_chembl_ID).filter(standard_type="IC50").only(['molecule_chembl_id','canonical_smiles','standard_value', 'standard_type'])
    df = pd.DataFrame(res)#Set a limit for now, take too long to compute over 500 activities [0:500]
    df['standard_value'] = df['standard_value'].astype(float)
    
    conditions = [
        (df['standard_value'] >= 10000),
        (df['standard_value'] <= 1000)
    ]
    choices = ['inactive', 'active']
    df['bioactivity_class'] = np.select(conditions, choices, default = 'intermediate')
    return(df)

In [5]:
df_bioactivities = get_bioactivities_data('CHEMBL612775')

In [6]:
df_bioactivities = df_bioactivities.dropna().reset_index(drop=True)

## Calculate Lipinski descriptors
Christopher Lipinski, a scientist at Pfizer, came up with a set of rule-of-thumb for evaluating the druglikeness of compounds. Such druglikeness is based on the Absorption, Distribution, Metabolism and Excretion (ADME) that is also known as the pharmacokinetic profile. Lipinski analyzed all orally active FDA-approved drugs in the formulation of what is to be known as the Rule-of-Five or Lipinski's Rule.

The Lipinski's Rule stated the following:

1. Molecular weight < 500 Dalton
2. Octanol-water partition coefficient (LogP) < 5
3. Hydrogen bond donors < 5
4. Hydrogen bond acceptors < 10

In [7]:
# Inspired by: https://codeocean.com/explore/capsules?query=tag:data-curation

def lipinski(smiles, verbose=False):

    moldata= []
    for elem in smiles:
        mol = Chem.MolFromSmiles(elem)
        moldata.append(mol)
       
    baseData= np.arange(1,1)
    i=0  
    for mol in moldata:        
       
        desc_MolWt = Descriptors.MolWt(mol)
        desc_MolLogP = Descriptors.MolLogP(mol)
        desc_NumHDonors = Lipinski.NumHDonors(mol)
        desc_NumHAcceptors = Lipinski.NumHAcceptors(mol)
           
        row = np.array([desc_MolWt,
                        desc_MolLogP,
                        desc_NumHDonors,
                        desc_NumHAcceptors])   
    
        if(i==0):
            baseData=row
        else:
            baseData=np.vstack([baseData, row])
        i=i+1      
    
    columnNames=["MW","LogP","NumHDonors","NumHAcceptors"]   
    descriptors = pd.DataFrame(data=baseData,columns=columnNames)
    
    return descriptors

In [8]:
df_lipinski = lipinski(df_bioactivities['canonical_smiles'])

In [9]:
df_combined = pd.concat([df_bioactivities, df_lipinski], axis=1)

## Convert IC50 to pIC50
To allow IC50 data to be more uniformly distributed, we will convert IC50 to the negative logarithmic scale which is essentially -log10(IC50).

This custom function pIC50() will accept a DataFrame as input and will:

Take the IC50 values from the standard_value column and converts it from nM to M by multiplying the value by 10
Take the molar value and apply -log10
Delete the standard_value column and create a new pIC50 column

In [10]:
# https://github.com/chaninlab/estrogen-receptor-alpha-qsar/blob/master/02_ER_alpha_RO5.ipynb

def pIC50(input):
    pIC50 = []

    for i in input['standard_value_norm']:
        molar = i*(10**-9) # Converts nM to M
        pIC50.append(-np.log10(molar))

    input['pIC50'] = pIC50
    x = input.drop('standard_value_norm', 1)
        
    return x

In [11]:
def norm_value(input):
    norm = []

    for i in input['standard_value']:
        if i > 100000000:
          i = 100000000
        norm.append(i)

    input['standard_value_norm'] = norm
    x = input.drop('standard_value', 1)
        
    return x

In [13]:
df_norm = norm_value(df_combined)


c:\Users\jacks\anaconda3\envs\drug_discovery\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  # Remove the CWD from sys.path while we load stuff.


In [14]:
df_final = pIC50(df_norm)

c:\Users\jacks\anaconda3\envs\drug_discovery\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  # This is added back by InteractiveShellApp.init_path()


In [15]:
df_final.to_csv('data/cleaned_bioactivity_data.csv', index=False)